In [1]:
import pandas as pd
import re
from pathlib import Path
import os
import numpy as np

pd.set_option('display.max_columns', 500)

In [10]:
STATUSES2PROCESS = ['Completed', 'Network Error']

In [2]:
DB_DIR = Path("C:/Playground/bhaiya-orders/zoho-bhor-farms/database/")

In [3]:
from ipyfilechooser import FileChooser

fc = FileChooser("C:/Playground/bhaiya-orders/zoho-bhor-farms/inputs/")
fc.use_dir_icons = True
display(fc)

FileChooser(path='C:/Playground/bhaiya-orders/zoho-bhor-farms/inputs/', filename='', show_hidden='False')

In [25]:
FILE = Path(fc.selected)

In [26]:
try:
    if 'csv' in FILE.suffix: df = pd.read_csv(FILE)
    else: df = pd.read_excel(FILE)
except OSError:
    print("Error occured, invalid file!")

In [27]:
df.head(3)

,Name,Select Drop Point,Select Building Ivory,Select Building Wellness Forever,Select Building NCP,Select Building Ajmera iLand,Select Building Sector 1/2,Select Building Cosmic,Select Wing,Flat No,Phone,Email,Select Delivery Date,Potatoes,Potato Price,Potato Cost Price,Select Quantity of Potatoes,Red Onions,Red Onions Price,Red Onions Cost Price,Select Quantity of Red Onions,Cauliflower,Cauliflower Price,Cauliflower Cost Price,Select Quantity of Cauliflower,Cabbage,Cabbage Price,Cabbage Cost Price,Select Quantity of Cabbage,Brinjal Bharta,Brinjal Bharta Price,Brinjal Bharta Cost Price,Select Quantity of Brinjal Bharta,Pumpkin,Pumpkin Price,Pumpkin Cost Price,Select Quantity of Pumpkin,Brinjal Kateri,Brinjal Kateri Price,Brinjal Kateri Cost Price,Select Quantity of Brinjal Kateri,Cucumber,Cucumber Price,Cucumber Cost Price,Select Quantity of Cucumber,Bhindi/Lady's Finger,Bhindi/Lady's Finger Price,Bhindi/Lady's Finger Cost Price,Select Quantity of Bhindi/Lady's Finger,Bitter Gourd,Bitter Gourd Price,Bitter Gourd Cost Price,Select Quantity of Bitter Gourd,Sponge Gourd,Sponge Gourd Price,Sponge Gourd Cost Price,Select Quantity of Sponge Gourd,Beetroot,Beetroot Price,Beetroot Cost Price,Select Quantity of Beetroot,Tondli,Tondli Price,Tondli Cost Price,Select Quantity of Tondli,Button Mushrooms,Button Mushroom Price,Button Mushroom Cost Price,Select Quantity of Button Mushroom,Raw Banana,Raw Banana Price,Raw Banana Cost Price,Select Quantity of Raw Banana,Bottle Gourd,Bottle Gourd Price,Bottle Gourd Cost Price,Select Quantity of Bottle Gourd,Drumsticks,Drumsticks Price,Drumsticks Cost Price,Select Quantity of Drumsticks,Capsicum,Capsicum Price,Capsicum Cost Price,Select Quantity of Capsicum,Gawar/Cluster Beans,Gawar/Cluster Beans Price,Gawar/Cluster Beans Cost Price,Select Quantity of Gawar/Cluster Beans,French Beans,French Beans Price,French Beans Cost Price,Select Quantity of French Beans,Baby Corn,Baby Corn Price,Baby Corn Cost Price,Select Quantity of Baby Corn,Tomato,Tomato Price,Tomato Cost Price,Select Quantity of Tomato,Cherry Tomatoes,Cherry Tomato Price,Cherry Tomato Cost Price,Select Quantity of Cherry Tomatoes,Sweet Corn,Sweet Corn Price,Sweet Corn Cost Price,Select Quantity of Sweet Corn,Broccoli,Broccoli Price,Broccoli Cost Price,Select Quantity of Broccoli,Lettuce,Lettuce Price,Lettuce Cost Price,Select Quantity of Lettuce,Apples Washington Per Kg,Apples Washington Price,Apples Washington Cost Price,Select Quantity of Apples Washington,VNR Guava,VNR Guava Price,VNR Guava Cost Price,Select Quantity of VNR Guava,Orange Imported,Imported Oranges Price,Imported Oranges Cost Price,Select Quantity of Imported Oranges,Pineapple Whole,Pineapple Whole Price,Pineapple Whole Cost Price,Select Quantity of Pineapple Whole,Kiwi Zespri Imported,Kiwi Zespri Imported Price,Kiwi Zespri Imported Cost Price,Select Quantity of Kiwi Zespri Imported,Banana Regular,Banana Price,Banana Cost Price,Select Quantity of Banana,Garlic,Garlic Price,Garlic Cost Price,Select Quantity of Garlic,GInger,Ginger Price,Ginger Cost Price,Select Quantity of Ginger,Green Chilly,Green Chilly Price,Green Chilly Cost Price,Select Quantity of Green Chilly,Lemons,Lemons Price,Lemons Cost Price,Select Quantity of Lemons,Curry Leaves,Curry Leaves Price,Curry Leaves Cost Price,Select Quantity of Curry Leaves,Coriander Leaves,Coriander Leaves Price,Coriander Leaves Cost Price,Select Quantity of Coriander Leaves,Mint Leaves,Mint Leaves Price,Mint Leaves Cost Price,Select Quantity of Mint Leaves,Lemongrass,Lemongrass Price,Lemongrass Cost Price,Select Quantity of Lemongrass,Palak/Spinach,Palak/Spinach Price,Palak/Spinach Cost Price,Select Quantity of Palak/Spinach,Shepu/Dill Leaves,Shepu/Dill Leaves Price,Shepu/Dill Leaves Cost Price,Select Quantity of Shepu/Dill Leaves,Methi Leaves,Methi Leaves Price,Methi Leaves Cost Price,Select Quantity of Methi Leaves,Red/Yellow Bell Peppers,Red/Yellow Bell Peppers Price,Red/Yellow Bell Peppers Cost Price,Select Quantity of Red/Ye

In [28]:
df['key'] = df['Name'] + ' ' + df['Added Time'].astype(str)

In [29]:
bldg_cols = []

for col in df.columns:
    if 'select building' in col.lower():
        bldg_cols.append(col)

df['building'] = df[bldg_cols].fillna('').sum(axis=1)

In [30]:
df['Payment Status'].unique()

array(['Completed', 'Network Error'], dtype=object)

In [31]:
df = df[df['Payment Status'].isin(STATUSES2PROCESS)]

In [32]:
# verification
df['Payment Status'].unique()

array(['Completed', 'Network Error'], dtype=object)

In [33]:
df = df.sort_values(['Select Drop Point', 'building', 'Select Wing'])
df['Token No'] = range(1, len(df)+1)

In [34]:
assert len(df) == len(df['key'].unique())

In [35]:
# extremely naive, needs a fix
all_items = [" ".join(i.split()[3:]) for i in df.columns if 'quantity of' in i.lower()]

In [36]:
items_dict = {}

In [37]:
def get_sell_price(item):
    val = df[item + ' ' + 'Price'].values[0]
    return val

def get_cost_price(item):
    val = df[item + ' ' + 'Cost Price'].values[0]
    return val

In [38]:
df_psize = pd.read_csv("C:/Playground/bhaiya-orders/zoho-bhor-farms/database/Packing Size.csv")
psize_dict = df_psize.set_index('Item').to_dict(orient='index')

In [39]:
def get_packing_size(item):
    if item in psize_dict.keys():
        return psize_dict[item]['Size']
    return -1 # => not supplied

def get_unit(item):
    if item in psize_dict.keys():
        return psize_dict[item]['Unit']
    return -1

In [65]:
import numpy as np

In [74]:
for item in all_items:
    dct = {}
    dct['packing_size'] = get_packing_size(item)
    dct['unit'] = get_unit(item)
#     if item == 'Cherry Tomatoes': item = 'Cherry Tomato'
#     if item == 'Potatoes': item = 'Potato'
    
    dct['sell_price'] = get_sell_price(item)
    dct['cost_price'] = get_cost_price(item)
    dct['date'] = df['Select Delivery Date'].unique()[0]
    
    items_dict[item] = dct

## Creating Items Database

In [47]:
# itemdb_df = pd.read_csv(DB_DIR/'items.csv')

In [76]:
# psizedb_df = pd.read_csv(DB_DIR/'item/psize.csv')

In [78]:
items_df = pd.DataFrame.from_dict(items_dict).T
items_df['date'] = pd.to_datetime(items_df.date.apply(lambda x: (x[:-2] + ' 2020')), format='%a %b %d %Y')
items_df.head()

,packing_size,unit,sell_price,cost_price,date
Potato,1,Kg,36,28,2020-08-24
Red Onions,1,Kg,18,12,2020-08-24
Cauliflower,1,Pieces,45,25,2020-08-24
Cabbage,1,Pieces,24,11,2020-08-24
Brinjal Bharta,1,Pieces,28,13,2020-08-24


### Psize Database

In [ ]:
# psizedb_df = pd.read_csv(DB_DIR/'item/psize.csv')

In [79]:
psizedb_df = items_df.drop(columns=['sell_price', 'cost_price'])

In [80]:
for dt in psizedb_df.date.unique():
    

,packing_size,unit,date
Potato,1,Kg,2020-08-24
Red Onions,1,Kg,2020-08-24
Cauliflower,1,Pieces,2020-08-24
Cabbage,1,Pieces,2020-08-24
Brinjal Bharta,1,Pieces,2020-08-24
Pumpkin,0.5,Kg,2020-08-24
Brinjal Kateri,0.5,Kg,2020-08-24
Cucumber,0.5,Kg,2020-08-24
Bhindi/Lady's Finger,0.5,Kg,2020-08-24
Bitter Gourd,0.3,Kg,2020-08-24


In [61]:
dtsrs[:4]

Potato         Mon Aug 24th
Red Onions     Mon Aug 24th
Cauliflower    Mon Aug 24th
Cabbage        Mon Aug 24th
Name: date, dtype: object

In [64]:
pd.to_datetime(dtsrs.apply(lambda x: (x[:-2] + ' 2020')), format='%a %b %d %Y')

Potato                    2020-08-24
Red Onions                2020-08-24
Cauliflower               2020-08-24
Cabbage                   2020-08-24
Brinjal Bharta            2020-08-24
Pumpkin                   2020-08-24
Brinjal Kateri            2020-08-24
Cucumber                  2020-08-24
Bhindi/Lady's Finger      2020-08-24
Bitter Gourd              2020-08-24
Sponge Gourd              2020-08-24
Beetroot                  2020-08-24
Tondli                    2020-08-24
Button Mushroom           2020-08-24
Raw Banana                2020-08-24
Bottle Gourd              2020-08-24
Drumsticks                2020-08-24
Capsicum                  2020-08-24
Gawar/Cluster Beans       2020-08-24
French Beans              2020-08-24
Baby Corn                 2020-08-24
Tomato                    2020-08-24
Cherry Tomato             2020-08-24
Sweet Corn                2020-08-24
Broccoli                  2020-08-24
Lettuce                   2020-08-24
Apples Washington         2020-08-24
V

In [42]:
df.fillna(0, inplace=True)

In [43]:
# create a mapping from item_id to the corresponding column to make sure no item goes unmapped

item2dropdowncol = {i : '' for i in all_items}

for index, row in df.iterrows():
    for col in df.columns:
        if 'quantity of' in col.lower():
            item2dropdowncol[" ".join(col.split()[3:])] = col
            
assert len(item2dropdowncol) == len(all_items)
for k, v in item2dropdowncol.items():
    assert len(v) > 3

In [44]:
def get_order(row):
    o = {}
    for item, qty_col in item2dropdowncol.items():
#         try:
        if row[qty_col] > 0:
            o[item] = row[qty_col]
#         except:
#             i = 'hello'
            
    return o

In [45]:
def get_total_cost_price(order):
    totalcp = 0
    for item, qty in order.items():
        if item == 'Cherry Tomatoes': item = 'Cherry Tomato'
        if item == 'Potatoes': item = 'Potato'
        totalcp += qty * get_cost_price(item)
    return totalcp

In [46]:
orders_dict = {}

for index, row in df.iterrows():
    dct = {}
    dct['name'] = row['Name']
    dct['drop_point'] = row['Select Drop Point']
    dct['building'] = row['building']
    dct['wing'] = row['Select Wing']
    dct['flat_no'] = row['Flat No']
    dct['phone_no'] = row['Phone']
    dct['email'] = row['Email']
    dct['delivery_date'] = row['Select Delivery Date']
    dct['total_amount'] = row['Total Amount']
    dct['payment_status'] = row['Payment Status']
    dct['order'] = get_order(row)
    dct['total_cost_price'] = get_total_cost_price(dct['order'])
    dct['profit'] = dct['total_amount'] - dct['total_cost_price']
    orders_dict[row['Token No']] = dct